# INTRO

* Conventional Multiplication
* Karatsuba Multiplication

In [1]:
import random
import numpy as np

## Conventional Multiplication

* Complexity: $O(n^2)$

In [220]:
# e.g.

#         23958233
#   ×         5830
#   ———————————————
#         00000000 ( =      23,958,233 ×     0)
#        71874699  ( =      23,958,233 ×    30)
#      191665864   ( =      23,958,233 ×   800)
#   + 119791165    ( =      23,958,233 × 5,000)
#   ———————————————
#     139676498390 ( = 139,676,498,390        )

In [98]:
def conv_mult(num1, num2, verbose=1):
    arr1, arr2 = [int(c) for c in str(num1)], [int(c) for c in str(num2)]
    len_arr1, len_arr2 = len(arr1), len(arr2)
    # cartesian-style mults
    arr = []
    for i in range(len_arr2-1,-1,-1): # every item in arr2 mult every item in arr1
        sub_arr = []
        carry = 0
        for j in range(len_arr1-1,-1,-1):
            ret = arr1[j]*arr2[i] + carry
            sub_arr.insert(0,ret%10)
            carry = ret/10
        if i>0:
            sub_arr.insert(0,carry)
        sub_arr += [0] * (len_arr2-1-i)
        arr.append(sub_arr)
    maxlen = max([len(sub_arr) for sub_arr in arr])
    arr = np.array([[0]*(maxlen-len(sub_arr))+sub_arr for sub_arr in arr])
    if verbose: print arr
    arr = arr.sum(axis=0)
    # summation
    carry = 0
    final = []
    for k in range(len(arr)-1,-1,-1):
        ret = arr[k]+carry
        final.insert(0,ret%10)
        carry = ret/10
    if carry!=0:
        final.insert(0,carry)
    return int(''.join([str(i) for i in final]))

In [99]:
conv_mult(5678,1234)

[[0 0 2 2 7 1 2]
 [0 1 7 0 3 4 0]
 [1 1 3 5 6 0 0]
 [5 6 7 8 0 0 0]]


7006652

In [107]:
%%timeit

conv_mult(random.getrandbits(200),random.getrandbits(200),verbose=0)

1000 loops, best of 3: 1.04 ms per loop


## Karatsuba Multiplication

* Complexity: $O(n^{log_23})$

Let $x,y$ be the factors in multiplication, say $x=1234,y=5678$, then we may split $x,y$ as follows:

$x: a = 12, b = 34;\quad y: c = 56, d = 78$

and write $x = 10^{n/2}a + b,\quad y = 10^{n/2}c + d$, then we have

$$\begin{align}x\times y &= (10^{n/2}a + b)\times(10^{n/2}c+d)\\ &= 10^n\cdot ac + 10^{n/2}(ad+bc) + bd\end{align}$$

In [218]:
def size_base10(num):
    return len(str(num))

def split_at(num, idx):
    return num/(10**idx), num%(10**idx)

def kara_mult(num1, num2):
    if (num1<10) or (num2<10):
        return num1*num2
    m = max(size_base10(num1), size_base10(num2))
    m2 = m/2
    a, b = split_at(num1, m2)
    c, d = split_at(num2, m2)
    ac = kara_mult(a, c)
    bd = kara_mult(b, d)
    z = kara_mult((a+b),(c+d))
    return (ac*(10**(2*m2))) + ((z-ac-bd)*(10**m2)) + bd

In [219]:
%%timeit

kara_mult(random.getrandbits(200),random.getrandbits(200))

1000 loops, best of 3: 705 µs per loop


In [223]:
x = 3141592653589793238462643383279502884197169399375105820974944592
y = 2718281828459045235360287471352662497757247093699959574966967627
print x; print
print y; print
print 'My Answer:'
print kara_mult(x, y); print
print 'Answer Key:'
print x*y; print 
print 'correct?', 'yes' if x*y==kara_mult(x,y) else 'no'

3141592653589793238462643383279502884197169399375105820974944592

2718281828459045235360287471352662497757247093699959574966967627

My Answer:
8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184

Answer Key:
8539734222673567065463550869546574495034888535765114961879601127067743044893204848617875072216249073013374895871952806582723184

correct? yes
